<h3> Imports </h3>


In [85]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import csv
import xml.etree.ElementTree as ET
import copy
import random
import uuid
import shutil

<h4>Preprocess script csv</h4>

In [86]:
# Function to convert time format to seconds
def time_to_seconds(time_str):
    time_obj=time_str.split(":")
    if len(time_obj)==2:
        return int(time_obj[0]) * 60 + int(time_obj[1])
    if len(time_obj)==3:
        return int(time_obj[0]) * 60 + int(time_obj[1])+ int(time_obj[2])/24
    else:
        print("time obj broken")
        return 0

In [87]:

# Function to merge consecutive rows with start times within 10 seconds
def merge_consecutive_rows(df):
    merged_rows = []
    current_row = None
    
    for index, row in df.dropna(how='all').iterrows():
        if current_row is None:
            current_row = row
        else:
            end_time_current = time_to_seconds(current_row['End Time'])
            #print(row['Start Time'])
            start_time_next = time_to_seconds(row['Start Time'])
            if start_time_next >= end_time_current and start_time_next - end_time_current <= 6:
                current_row['End Time'] = row['End Time']
                current_row['Transcript'] += ' ' + row['Transcript']
            else:
                current_row['Duration'] = convert_time_to_frames(current_row['End Time'], 24) - convert_time_to_frames(current_row['Start Time'], 24)
                merged_rows.append(current_row)
                
                current_row = row
    
    if current_row is not None:
        current_row['Duration'] = convert_time_to_frames(current_row['End Time'], 24) - convert_time_to_frames(current_row['Start Time'], 24)
        merged_rows.append(current_row)
    
    merged_df = pd.DataFrame(merged_rows)
    return merged_df

# script_df = merge_consecutive_rows(scriptCsv)
# print(script_df)

<h3>build xml structure</h3>

In [88]:

def create_xml(xml_name):
    
    # Create the root element
    root = ET.Element("xmeml", version="4")
    
    # Create the sequence element with attributes
    sequence = ET.SubElement(root, "sequence", id="sequence-2", TL_SQAudioVisibleBase="0", TL_SQVideoVisibleBase="0",
                             TL_SQVisibleBaseTime="0", TL_SQAVDividerPosition="0.5", TL_SQHideShyTracks="0",
                             TL_SQHeaderWidth="236", TL_SQDataTrackViewControlState="0",
                             Monitor_ProgramZoomOut="340011984312000", Monitor_ProgramZoomIn="0",
                             TL_SQTimePerPixel="1.6034289012958367", MZ_EditLine="333083126376000",
                             MZ_Sequence_PreviewFrameSizeHeight="1080", MZ_Sequence_PreviewFrameSizeWidth="1920",
                             MZ_Sequence_AudioTimeDisplayFormat="200", MZ_Sequence_PreviewUseMaxRenderQuality="false",
                             MZ_Sequence_PreviewUseMaxBitDepth="false", MZ_Sequence_VideoTimeDisplayFormat="110",
                             MZ_WorkOutPoint="15235011792000", MZ_WorkInPoint="0", MZ_ZeroPoint="0", explodedTracks="true")
    
    # Add the uuid element
    uuid = ET.SubElement(sequence, "uuid")
    uuid.text = "50e61931-251f-4069-8193-a3fbad7f93ff"
    
    # Add the duration element
    duration = ET.SubElement(sequence, "duration")
    duration.text = "31533"
    
    # Add the rate element with nested timebase and ntsc elements
    rate = ET.SubElement(sequence, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    
    # Add the name element
    name_element = ET.SubElement(sequence, "name")
    name_element.text = xml_name
    
    # Add the media element with nested video and audio elements
    media = ET.SubElement(sequence, "media")

    
    # Add the timecode element with nested rate, string, frame, and displayformat elements
    timecode = ET.SubElement(sequence, "timecode")
    rate = ET.SubElement(timecode, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    string = ET.SubElement(timecode, "string")
    string.text = "00:00:00:00"
    frame = ET.SubElement(timecode, "frame")
    frame.text = "0"
    displayformat = ET.SubElement(timecode, "displayformat")
    displayformat.text = "NDF"
    
    # Add the labels element with nested label2 element
    labels = ET.SubElement(sequence, "labels")
    label2 = ET.SubElement(labels, "label2")
    label2.text = "Forest"
    

    # Create the ElementTree object with the root element
    tree = ET.ElementTree(root)
    
    # Generate a random Idd
    idd = str(random.randint(1, 1000))
    
    # Save the XML to a file
    filename = f"{xml_name}-{idd}.xml"
    tree.write(f"{filename}", encoding="utf-8", xml_declaration=True)
    print(f"XML saved to {filename}")

    return filename 



<h4> try to refeactor these away</h4>

In [89]:
extract_method = 'project_timecode'  # video_timecode or project_timecode 

id_mapping = {}
# global sequence_start
# global sequence_end
subtitle_counter=1
sequence_start=0
sequence_end=0
audio_sequence_start=0
audio_sequence_end=0
xml_file_assigned=[]

<h3> Helper functions for XML generation </h3>

Converts Time to Frames 

In [90]:
def convert_time_to_frames(time, rate):
    # print('Time', time, 'Rate', rate)
    parts = time.split(":")
    
    hours=0    #If they aren't defined, they are assumed to be 0
    frames=0
    
    if len(parts)==4:
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        frames = int(parts[3])

    elif len(parts)==3:
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        
    elif len(parts)==2:
        minutes = int(parts[0])
        seconds = int(parts[1])
        
        # Calculate the total duration in frames
    total_frames = (hours * 3600 * rate +  minutes * 60 * rate + seconds * rate)
    
    return total_frames

convert_time_to_frames('00:54:36:38', 24)

78624

In [91]:
convert_time_to_frames("1:00",24)

1440

In [92]:
def frame_to_ticks(frame_number, frame_rate):
    total_ticks_in_a_second = 254016000000
    tick_value = int((frame_number * total_ticks_in_a_second) / frame_rate)
    return tick_value

# frame_to_ticks(24, 24)

<h3> XML processing functions </h3>

Generate New ids for the each Clip item in seperate files

In [93]:
def generate_unique_id():
    unique_id = f"clipitem-{uuid.uuid4().hex}"
    return unique_id

generate_unique_id()

'clipitem-28319c896aa9462191015b4ea82b9306'

Update the id of clips in specific XML file

In [94]:
def update_xml_ids(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    print('Assigning unique IDs to clipitem elements...')

    # Generate unique IDs for clipitem elements
      # Store the mapping of old IDs to new unique IDs
    for clip_item in root.findall('.//clipitem'):  # Adjust the XPath expression based on the actual location of clipitem elements
        current_id = clip_item.attrib['id']
        unique_id = generate_unique_id()
        id_mapping[current_id] = unique_id
        clip_item.attrib['id'] = unique_id

    
        # Update the references in the links section
    for link in root.findall('.//link'):
        linkclipref = link.find('linkclipref')
        if linkclipref is not None and linkclipref.text in id_mapping:
            linkclipref.text = id_mapping[linkclipref.text]
    #print(id_mapping)

    # Save the modified XML file
    tree.write(xml_file, encoding="utf-8")

# update_xml_ids(xml_file)

Extract time in  start and end time format

In [95]:
def extract_timecode(timecode_range):
    start_time, end_time = timecode_range.split(" - ")
    start_time = start_time.strip().replace(" ", "")
    end_time = end_time.strip().replace(" ", "")
    # print('Start Time',start_time,'End Time', end_time)
    return start_time, end_time

extract_timecode("00:00:00:00 - 00:23:00:00")

('00:00:00:00', '00:23:00:00')

<h3> Extract sequences </h3>

Extract sequence info through xml file and extract the clips that match with the start and end time using frame comparison

In [96]:
###this isn't being used at the moment, but it looks like this could be a good function to integrate into the createXml if needed (although the sequence settings are so standardized this doesn't seem needed)

def extract_sequence_info(xml_file, start_time, end_time):
    global sequence_start, sequence_end
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extract sequence information
    sequence = root.find("sequence")
    sequence_info = {
        "duration": int(sequence.find("duration").text),
        "rate": {
            "timebase": int(sequence.find("rate/timebase").text),
            "ntsc": sequence.find("rate/ntsc").text == "TRUE",
        },
    }
    sequence_rate = sequence_info["rate"]["timebase"]

    # Convert start and end times to frames
    start_frame = convert_time_to_frames(start_time, 24)
    end_frame = convert_time_to_frames(end_time, 24)
    proTickIn = frame_to_ticks(start_frame, 24)
    proTickOut = frame_to_ticks(end_frame, 24)

    total_clip_frames = end_frame - start_frame
    total_clip_proticks = proTickOut - proTickIn

    # print('Start and End Frame',start_frame ,end_frame)
    # print(sequence_rate,'sequence_rate')

   

In [97]:

# ###check if this was refactored into extract_video_sequence function

#     # Extract video clip information
#     video_clips = []
#     for track_index, track in enumerate(root.findall(".//video/track")):  # Iterate over all tracks within the <video> tag
#         # track_idx = 0
#         track_targeted = track.attrib["MZ.TrackTargeted"]
#         for clip_item in track.findall("clipitem"):  # Only consider clip items within the <video> tag
            
#             video_start_cut_diff=abs(start_frame - int(clip_item.find("start").text))
#             video_end_cut_diff=abs(end_frame - int(clip_item.find("end").text))
#             extracted_clip_duration=end_time-start_time
#             actual_clip_duration=int(clip_item.find("duration").text)


#             links = clip_item.findall("link")
#             track_idx = [
#                 int(link.find("trackindex").text)
#                 for link in clip_item.findall("link")
#                 if link.find("mediatype").text == "video"
#             ]
#             print("Track Index", track_idx)
#             print("Clip ID", clip_item.attrib["id"])

#             clip_info = {
#                 "id": clip_item.attrib["id"],  # Get the clip ID
#                 "name": clip_item.find("name").text,
#                 "duration": int(clip_item.find("duration").text),
#                 "rate": {
#                     "timebase": int(clip_item.find("rate/timebase").text),
#                     "ntsc": clip_item.find("rate/ntsc").text == "TRUE",
#                 },
#                 "in": int(clip_item.find("in").text),
#                 "out": int(clip_item.find("out").text),
#                 "start": int(clip_item.find("start").text),
#                 "end": int(clip_item.find("end").text),
#                 "track_index": track_index+1,
#                 "MZ_TrackTargeted": track_targeted,
#                 "links": [],  # Initialize an empty list to store links,
#                 "video_clip_element": None,  # Store the clip item element for later use
#                 "linked_audio_clip_elements_list": [],  # Initialize an empty list to store linked clip items
#             }

#             if extract_method == "video_timecode":
#                 clip_comparison = (
#                     clip_info["in"] <= start_frame <= clip_info["out"]
#                 ) or (clip_info["in"] <= end_frame <= clip_info["out"])
#             else:
#                 clip_comparison = (
#                     clip_info["start"] <= start_frame <= clip_info["end"]
#                 ) or (clip_info["start"] <= end_frame <= clip_info["end"])

            

#             for link in links:
#                 link_info = {
#                     "linkclipref": link.find("linkclipref").text,
#                     "mediatype": link.find("mediatype").text,
#                     "trackindex": int(link.find("trackindex").text),
#                     "clipindex": int(link.find("clipindex").text),
#                 }
#                 if link.find("groupindex") is not None:
#                     link_info["groupindex"] = int(link.find("groupindex").text)
#                 clip_info["links"].append(link_info)
#                 if link.find("mediatype").text == "audio":
#                     audio_clip_items = root.findall(".//audio//clipitem")
#                     for audio_clip_item in audio_clip_items:
#                         if (
#                             audio_clip_item.attrib["id"]
#                             == link.find("linkclipref").text
#                         ):
#                             audio_clip_item.find("in").text = str(start_frame)
#                             audio_clip_item.find("out").text = str(end_frame)
                              
#                             audio_clip_item.find("start").text = str(sequence_start)
#                             audio_clip_item.find("end").text = str(sequence_end)
#                             audio_clip_item.find("pproTicksIn").text = str(proTickIn)
#                             audio_clip_item.find("pproTicksOut").text = str(proTickOut)
                                
                            
#                             clip_info["linked_audio_clip_elements_list"].append(
#                                 {
#                                     "audio_clip_item": audio_clip_item,
#                                     "trackindex": int(link.find("trackindex").text),
#                                     "sourceindex": int(
#                                         audio_clip_item.find(
#                                             ".//sourcetrack/trackindex"
#                                         ).text
#                                     ),
#                                 }
#                             )

#             # Check if the clip's in or out frame falls within the given start and end frames
#             # print('Clip Frame Inside file Found: ',clip_info['in'] <= start_frame <= clip_info['out'])
#             print(
#                 "Clip Frame Inside file Found: ",
#                 clip_info["start"] <= start_frame <= clip_info["end"]
#                 or clip_info["start"] <= end_frame <= clip_info["end"],
#             )
#             print("clip_comparison", clip_comparison)
#             print("In - Matching - Out")
#             # print(clip_info['in'],start_frame,clip_info['out'])
#             print(clip_info["start"], start_frame, clip_info["end"])
#             print(clip_info["start"], end_frame, clip_info["end"])
#             if clip_comparison:
#                 clip_item.find("in").text = str(start_frame)
#                 clip_item.find("out").text = (
#                     str(end_frame)
#                     if extract_method == "video_timecode"
#                     else str(start_frame + total_clip_frames)
#                 )
#                 clip_item.find("start").text = str(sequence_start)
#                 clip_item.find("end").text = str(sequence_end)
#                 clip_item.find("pproTicksIn").text = str(proTickIn)
#                 clip_item.find("pproTicksOut").text = (
#                     str(proTickOut)
#                     if extract_method == "video_timecode"
#                     else str(proTickIn + total_clip_proticks)
#                 )
#                 clip_info["video_clip_element"] = clip_item
#                 video_clips.append(clip_info)
#                 print("Next Sequence Start", sequence_start)
#                 print("Next Sequence End", sequence_end)

#     # print(clip_info, start_frame, end_frame)

#     # Create result dictionary
#     result = { "video_clips": video_clips}
#     # print('Result',result)

#     if not video_clips:  # Check if video_clips list is empty
#         return None

    
#      return result

Overlap Function

Extract Video sequences from the xml file

In [98]:
def extract_video_sequence(xml_file,row,offset_time):
    print(row)
    #these start and end times are from the transcripts
    start_time=row["Start Time"]
    end_time=row["End Time"]
    
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Convert start and end times to frames
    start_frame = convert_time_to_frames(start_time, 24)
    end_frame = convert_time_to_frames(end_time, 24)


    # Extract video clip information
    video_clips = []
    
    for track_index, track in enumerate(root.findall(".//video/track")):  # Iterate over all tracks within the <video> tag
        track_targeted = track.attrib["MZ.TrackTargeted"]
        for clip_item in track.findall("clipitem"):  # Only consider clip items within the <video> 

            clip_info = {
                "id": clip_item.attrib["id"],  # Get the clip ID
                "new_id": generate_unique_id(),  # Generate a new unique ID for the clip item",
                "name": clip_item.find("name").text,
                "duration": int(clip_item.find("duration").text),
                "rate": {
                    "timebase": int(clip_item.find("rate/timebase").text),
                    "ntsc": clip_item.find("rate/ntsc").text == "TRUE",
                },
                "in": int(clip_item.find("in").text),
                "out": int(clip_item.find("out").text),
                "start": int(clip_item.find("start").text),
                "end": int(clip_item.find("end").text),
                "track_index": track_index+1,
                "MZ_TrackTargeted": track_targeted,
                "video_clip_element": None,  # Store the clip item element for later use,
                "type":"video",
                "text":row["Transcript"],
                "interview":row["Interview"],
                "speaker":row["Speaker"],
                "duration":row["Duration"]
            }

            # Check if the clip's in or out frame falls within the given start and end frames
            if (
                clip_info["start"] <= start_frame <= clip_info["end"]
                or clip_info["start"] <= end_frame <= clip_info["end"]
            ):
                new_cutoff_left= start_frame - clip_info["start"]                 
                if new_cutoff_left<0:
                    new_cutoff_left=0
                
                new_cutoff_right= clip_info["end"]-end_frame
                if new_cutoff_right<0:
                    new_cutoff_right=0

                new_in = clip_info["in"] + new_cutoff_left
                new_out = clip_info["out"] - new_cutoff_right
                new_start = int(clip_info["start"]-start_frame)
                if new_start<0:
                    new_start=0
                new_start+=offset_time
                new_end = new_start+(new_out-new_in)
                
                
                #print(offset_time,new_in,new_out,new_start,new_end,row["Transcript"])
                
                # Update the clip's in, out, start, and end in the XML tree based on the overlapped segment
                clip_item.find("in").text = str(new_in) #+ round(ratio_start * (clip_info["out"] - clip_info["in"])))
                clip_item.find("out").text = str(new_out) #+ round(ratio_end * (clip_info["out"] - clip_info["in"])))
                clip_item.find("start").text = str(new_start)
                clip_item.find("end").text = str(new_end)
                
                #these aren't used, just for the csv
                # Update the clip's in, out, start, and end in the XML tree based on the overlapped segment
                clip_info["in"] = str(new_in) #+ round(ratio_start * (clip_info["out"] - clip_info["in"])))
                clip_info["out"] = str(new_out) #+ round(ratio_end * (clip_info["out"] - clip_info["in"])))
                clip_info["start"] = str(new_start)
                clip_info["end"] = str(new_end)
                

                clip_info["video_clip_element"] = clip_item
                #print(clip_info)

                video_clips.append(clip_info)
                
    return video_clips

# Test the function with the given XML file and start/end times
# extract_video_sequence('ben - synced.xml', '00:00:00:00', '00:00:10:00')


Extract Audio sequences from the xml file

In [99]:
def extract_audio_sequence(xml_file, row,offset_time):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    start_time=row["Start Time"]
    end_time=row["End Time"]

    global audio_sequence_start ,audio_sequence_end

    # Convert start and end times to frames
    start_frame = convert_time_to_frames(start_time, 24)
    end_frame = convert_time_to_frames(end_time, 24)

    # Extract audio clip information
    audio_clips = []
    for track_index, track in enumerate(root.findall(".//audio/track")):  # Iterate over all tracks within the <video> tag
        track_targeted = track.attrib["MZ.TrackTargeted"]
        currentExplodedTrackIndex = track.attrib["currentExplodedTrackIndex"]
        for clip_item in track.findall("clipitem"):  # Only consider clip items within the <video> tag
            clip_info = {
                "id": clip_item.attrib["id"],  # Get the clip ID
                "name": clip_item.find("name").text,
                "duration": int(clip_item.find("duration").text),
                "rate": {
                    "timebase": int(clip_item.find("rate/timebase").text),
                    "ntsc": clip_item.find("rate/ntsc").text == "TRUE",
                },
                "in": int(clip_item.find("in").text),
                "out": int(clip_item.find("out").text),
                "start": int(clip_item.find("start").text),
                "end": int(clip_item.find("end").text),
                "track_index": track_index+1,
                "MZ_TrackTargeted": track_targeted,
                "currentExplodedTrackIndex": currentExplodedTrackIndex,
                #"audio_clip_element": None,  # Store the clip item element for later use
                "sourceindex": int(clip_item.find(".//sourcetrack/trackindex").text),
                "type":"audio",
                "text":row["Transcript"],
                "interview":row["Interview"],
                "speaker":row["Speaker"],
            }
            

            # Check if the clip's in or out frame falls within the given start and end frames
            if (
                clip_info["start"] <= start_frame <= clip_info["end"]
                or clip_info["start"] <= end_frame <= clip_info["end"]
            ):
               
                new_cutoff_left= start_frame - clip_info["start"]
                if new_cutoff_left<0:
                    new_cutoff_left=0
                
                new_cutoff_right= clip_info["end"]-end_frame
                if new_cutoff_right<0:
                    new_cutoff_right=0

                new_in = clip_info["in"] + new_cutoff_left
                new_out = clip_info["out"] - new_cutoff_right
                new_start = int(clip_info["start"]-start_frame)
                if new_start<0:
                    new_start=0
                new_start+=offset_time
                new_end = new_start+(new_out-new_in)
                
                                
                # Update the clip's in, out, start, and end in the XML tree based on the overlapped segment
                clip_item.find("in").text = str(new_in) #+ round(ratio_start * (clip_info["out"] - clip_info["in"])))
                clip_item.find("out").text = str(new_out) #+ round(ratio_end * (clip_info["out"] - clip_info["in"])))
                clip_item.find("start").text = str(new_start)
                clip_item.find("end").text = str(new_end)
                
                #these aren't used, just for the csv
                # Update the clip's in, out, start, and end in the XML tree based on the overlapped segment
                clip_info["in"] = str(new_in) #+ round(ratio_start * (clip_info["out"] - clip_info["in"])))
                clip_info["out"] = str(new_out) #+ round(ratio_end * (clip_info["out"] - clip_info["in"])))
                clip_info["start"] = str(new_start)
                clip_info["end"] = str(new_end)


                clip_item.find("pproTicksIn").text = str(frame_to_ticks(new_in, 24))
                clip_item.find("pproTicksOut").text = str(frame_to_ticks(new_out, 24))

                clip_info["audio_clip_element"] = clip_item
                audio_clips.append(clip_info)

    # Create result dictionary
    return audio_clips



These functions go through all the xml files and call the  extract clips function

In [100]:
def process_csv_file(csv_file,project_name):

    video_clips=[]
    audio_clips=[]
    
    dfList=[]
    
    offset_time=0
    for index, row in csv_file.iterrows():

        xml_file = os.path.join( "../03_Interview XML/"+project_name, row["Interview"]+" - final.xml")

        #the call here to extract_video/audio_sequences is where the magic happens
        new_video_clips=extract_video_sequence(xml_file, row,offset_time)
        new_audio_clips=extract_audio_sequence(xml_file, row,offset_time)
        dfList+=new_audio_clips
        dfList+=new_video_clips
        offset_time+=row["Duration"]
    
    df=pd.DataFrame.from_records(dfList)
    #print("result",result)
    return df
            

Append Video to XML File

In [101]:
def append_video_to_xml(file_name, xml_data):
    print(f"Adding Video Tracks to {file_name}")
    # print(xml_json_data)
    # Parse the XML data
    tree = ET.parse(file_name)
    root = tree.getroot()

    # Find the media element
    media = root.find('.//media')

    # Append the provided code inside the media element
    video = ET.SubElement(media, "video")

    # Add the format element with nested samplecharacteristics element
    format_ = ET.SubElement(video, "format")
    samplecharacteristics = ET.SubElement(format_, "samplecharacteristics")
    
    # Add the rate element with nested timebase and ntsc elements inside samplecharacteristics
    rate = ET.SubElement(samplecharacteristics, "rate")
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "24"
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "TRUE"
    
    # Add the codec element with nested name and appspecificdata elements
    codec = ET.SubElement(samplecharacteristics, "codec")
    name = ET.SubElement(codec, "name")
    name.text = "Apple ProRes 422"
    appspecificdata = ET.SubElement(codec, "appspecificdata")
    
    # Add the appname, appmanufacturer, and appversion elements inside appspecificdata
    appname = ET.SubElement(appspecificdata, "appname")
    appname.text = "Final Cut Pro"
    appmanufacturer = ET.SubElement(appspecificdata, "appmanufacturer")
    appmanufacturer.text = "Apple Inc."
    appversion = ET.SubElement(appspecificdata, "appversion")
    appversion.text = "7.0"
    
    # Add the data element with nested qtcodec element inside appspecificdata
    data = ET.SubElement(appspecificdata, "data")
    qtcodec = ET.SubElement(data, "qtcodec")
    codecname = ET.SubElement(qtcodec, "codecname")
    codecname.text = "Apple ProRes 422"
    codectypename = ET.SubElement(qtcodec, "codectypename")
    codectypename.text = "Apple ProRes 422"
    codectypecode = ET.SubElement(qtcodec, "codectypecode")
    codectypecode.text = "apcn"
    codecvendorcode = ET.SubElement(qtcodec, "codecvendorcode")
    codecvendorcode.text = "appl"
    spatialquality = ET.SubElement(qtcodec, "spatialquality")
    spatialquality.text = "1024"
    temporalquality = ET.SubElement(qtcodec, "temporalquality")
    temporalquality.text = "0"
    keyframerate = ET.SubElement(qtcodec, "keyframerate")
    keyframerate.text = "0"
    datarate = ET.SubElement(qtcodec, "datarate")
    datarate.text = "0"
    
    # Add the width, height, anamorphic, pixelaspectratio, fielddominance, and colordepth elements inside samplecharacteristics
    width = ET.SubElement(samplecharacteristics, "width")
    width.text = "1920"
    height = ET.SubElement(samplecharacteristics, "height")
    height.text = "1080"
    anamorphic = ET.SubElement(samplecharacteristics, "anamorphic")
    anamorphic.text = "FALSE"
    pixelaspectratio = ET.SubElement(samplecharacteristics, "pixelaspectratio")
    pixelaspectratio.text = "square"
    fielddominance = ET.SubElement(samplecharacteristics, "fielddominance")
    fielddominance.text = "none"
    colordepth = ET.SubElement(samplecharacteristics, "colordepth")
    colordepth.text = "24"
    
    video_tracks = []   
    for index,clip in xml_data.iterrows():
        video_tracks.append({'MZ_TrackTargeted':clip["MZ_TrackTargeted"],'track_index':clip["track_index"]})
    #print("preDict",video_tracks)

    # Make video_tracks unique based on track_index
    video_tracks = [dict(t) for t in {tuple(d.items()) for d in video_tracks}]
    #print(video_tracks)

    # Create video track elements and append video clips
    for track in video_tracks:
        video_track = ET.SubElement(video, "track", TL_SQTrackShy="0", TL_SQTrackExpandedHeight="25",
                                    TL_SQTrackExpanded="0", MZ_TrackTargeted=f"{track['MZ_TrackTargeted']}")
        track_data=xml_data[xml_data["track_index"]==track['track_index']]
        for index,row in track_data.iterrows():
            video_track.append(copy.deepcopy(row["video_clip_element"]))

    # Save the modified XML to a new file
    modified_tree = ET.ElementTree(root)
    modified_tree.write(file_name)    




Append Audio to XML File

In [102]:

def append_audio_to_xml(file_name, xml_data):
    print(f"Adding Audio Tracks to {file_name}")
    # Parse the XML data
    audio_tracks = []
    track_index = []
    source_index = []
    for index,audio_clip in xml_data.iterrows():
        if audio_clip['currentExplodedTrackIndex'] not in track_index:
            track_index.append(audio_clip['currentExplodedTrackIndex'])
            source_index.append(audio_clip['sourceindex'])
            audio_tracks.append({'source_index': audio_clip['sourceindex'],
                                 'MZ_TrackTargeted': audio_clip['MZ_TrackTargeted'],
                                 'currentExplodedTrackIndex':audio_clip['currentExplodedTrackIndex']})

    audio_track_count = len(set(source_index))

    # print('audio_tracks',audio_tracks)
    # print('track_index',track_index)
    # print('source_index',source_index)
    # print('audio_track_count',audio_track_count)
    
    tree = ET.parse(file_name)
    root = tree.getroot()

    # Find the media element
    media = root.find('.//media')

    # Add the audio element inside media
    audio = ET.SubElement(media, "audio")

    # constant stuff

    # Create subelements and append them to the audio element
    num_output_channels = ET.SubElement(audio, 'numOutputChannels')
    num_output_channels.text = '2'

    format_element = ET.SubElement(audio, 'format')
    sample_characteristics = ET.SubElement(format_element, 'samplecharacteristics')
    depth = ET.SubElement(sample_characteristics, 'depth')
    depth.text = '16'
    sample_rate = ET.SubElement(sample_characteristics, 'samplerate')
    sample_rate.text = '48000'

    outputs = ET.SubElement(audio, 'outputs')

    group_1 = ET.SubElement(outputs, 'group')
    index_1 = ET.SubElement(group_1, 'index')
    index_1.text = '1'
    num_channels_1 = ET.SubElement(group_1, 'numchannels')
    num_channels_1.text = '1'
    downmix_1 = ET.SubElement(group_1, 'downmix')
    downmix_1.text = '0'
    channel_1 = ET.SubElement(group_1, 'channel')
    channel_index_1 = ET.SubElement(channel_1, 'index')
    channel_index_1.text = '1'

    group_2 = ET.SubElement(outputs, 'group')
    index_2 = ET.SubElement(group_2, 'index')
    index_2.text = '2'
    num_channels_2 = ET.SubElement(group_2, 'numchannels')
    num_channels_2.text = '1'
    downmix_2 = ET.SubElement(group_2, 'downmix')
    downmix_2.text = '0'
    channel_2 = ET.SubElement(group_2, 'channel')
    channel_index_2 = ET.SubElement(channel_2, 'index')
    channel_index_2.text = '2'

    
     # Create audio track elements and append audio clips
    for audio_track_index in audio_tracks:
        #print(audio_track_index)
        audio_track = ET.SubElement(audio, "track", TL_SQTrackAudioKeyframeStyle="0", TL_SQTrackShy="0",
                                    TL_SQTrackExpandedHeight="25", TL_SQTrackExpanded="0",
                                    MZ_TrackTargeted=f"{audio_track_index['MZ_TrackTargeted']}", PannerCurrentValue="0.5", PannerIsInverted="true",
                                    PannerStartKeyframe="-91445760000000000,0.5,0,0,0,0,0,0", PannerName="Balance",
                                    currentExplodedTrackIndex=f"{audio_track_index['currentExplodedTrackIndex']}", totalExplodedTrackCount=f"{audio_track_count}",
                                    premiereTrackType="Stereo")

        for index,audio_clip in xml_data.iterrows():
            if int(audio_track_index['currentExplodedTrackIndex']) == int(audio_clip["currentExplodedTrackIndex"]):
                audio_track.append(copy.deepcopy(audio_clip["audio_clip_element"]))
                   # Save the modified XML to a new file
    modified_tree = ET.ElementTree(root)
    modified_tree.write(file_name)


   

<h2> Run the processing for a script </h2>

In [103]:

# the run_extraction() function serves as the main entry point for running the extraction process. 
# It calls functions to process a CSV file, filter the resulting list of clips, 
# and creates an XML file with the audio and video from the filtered clips

def run_extraction(project_id,script_name): 
    
    ###open csv and preprocess, merging consecutive rows
    script_csv=pd.read_csv("../01_Scripts/"+project_id+"/CSV/"+script_name+".csv")
    script_df = merge_consecutive_rows(script_csv) ##note this is generated by running the Docx > CSV > SRT Pipeline. should it be brought here?
    print(script_df)
    
    #iterate through csv, ## extract audio and video sequence info, ## add audio, video, and subtitle info to xml
    xml_pieces=process_csv_file(script_df,project_id)##note this also goes into the xml files as well. Could consider refactor here to have an intermediate product
    xml_pieces.to_csv("../09_Temp/xml_pieces_test.csv")##temp output
    #print("xml_pieces",xml_pieces)
    
    export_xml_path="../06_Export XMLs/"+project_id+"/Archive/"+script_name
    filename = create_xml(export_xml_path)##note that the xml template is generated here, and the filename is returned
    append_video_to_xml(filename, xml_pieces[xml_pieces["type"]=="video"])
    append_audio_to_xml(filename, xml_pieces[xml_pieces["type"]=="audio"])
    shutil.copyfile(filename, "../06_Export XMLs/"+project_id+"/"+script_name+".xml")

    
project_id="Test"
script_name="wolunka-test"
    
run_extraction(project_id,script_name)


    Speaker Interview Start Time End Time  Duration (Seconds)  \
0   Magalys   wolunka      01:00    01:21            4.380952   
5   Magalys   wolunka      00:24    00:42            3.802817   
10  Magalys   wolunka      00:01    00:19            3.701923   
15  Magalys   wolunka      00:44    00:58            4.000000   

                                           Transcript  Duration  
0   I am glad to know the place where Wolunka was....       504  
5   TRUE. In fact, in our culture it is necessary ...       432  
10  Well Neko, you are entering the Macuira mounta...       432  
15  That's why I'm here in your territory Magalys....       336  
Speaker                                                         Magalys
Interview                                                       wolunka
Start Time                                                        01:00
End Time                                                          01:21
Duration (Seconds)                                      

In [104]:

# ### Update these to run this pipeline over a project and script
# project_id = "Wayuu"
# script_name="magalys test"

# run_extraction(project_id,script_name)